<div align="center">

# Investissement en une fois vs Investissement Progressif (DCA)
#### Faut-il investir tout d'un coup ou répartir dans le temps ?

<br>

</div>

Vous venez de recevoir une somme importante — une prime, un héritage, ou des économies que vous avez enfin décidé d'investir. La question classique se pose : **faut-il tout investir immédiatement, ou répartir l'investissement dans le temps ?**

La première approche est l'**investissement en une fois (Lump Sum)** — tout mettre dès le premier jour. La seconde est le **Dollar Cost Averaging (DCA)** — investir des montants fixes à intervalles réguliers, par exemple 1 000 € par mois pendant un an.

Ce notebook vous permet d'explorer les deux stratégies en utilisant 53 ans de données de marché réelles. Vous verrez quelle stratégie gagne le plus souvent, de combien, et pourquoi la réponse n'est pas aussi simple que « faites toujours X ».

*Note : Cette analyse utilise des données d'actions mondiales (indice MSCI World). Les résultats peuvent différer pour les obligations ou d'autres classes d'actifs.*

In [ ]:
%pip install -q ipywidgets==8.1.8 ipecharts==1.4.0 pandas==2.3.3 numpy==2.4.0

In [ ]:
import datetime
import numpy as np
import pandas as pd
from IPython.display import display, clear_output
from ipywidgets import (
    HTML,
    HBox,
    VBox,
    Layout,
    Button,
    Output,
    FloatText,
    DatePicker,
    IntText,
    Dropdown,
    IntProgress,
)
from ipecharts import EChartsRawWidget
from investment_utils import (
    BacktestParams,
    InvestmentResult,
    calculate_lump_sum,
    calculate_dca,
    run_multiple_backtests,
)
from market_data import load_msci_world

In [ ]:
COLORS = {
    "lump_sum": "#73c0de",
    "dca": "#ee6666",
    "success": "#3fa266",
    "danger": "#e34671",
}

# Fix button text alignment and add responsive card styles
display(HTML("""
<style>
.widget-button {
    display: flex !important;
    align-items: center !important;
    justify-content: center !important;
}
.widget-button i {
    margin-right: 6px;
}
/* Fix dropdown label cropping */
.widget-dropdown select {
    height: auto !important;
}
/* Stat cards (backtest results) */
.stat-cards-container {
    display: flex;
    flex-direction: row;
    gap: 20px;
    margin: 20px 0;
    justify-content: center;
    flex-wrap: wrap;
}
.stat-card {
    background: var(--jp-layout-color1, #f5f5f5);
    padding: 20px;
    border-radius: 8px;
    text-align: center;
    flex: 1 1 150px;
    min-width: 140px;
    max-width: 220px;
    box-sizing: border-box;
    border: 1px solid var(--jp-border-color2, rgba(0, 0, 0, 0.12));
    display: flex;
    flex-direction: column;
    justify-content: center;
}
/* Strategy cards (comparison results) */
.strategy-cards-container {
    display: flex;
    flex-direction: row;
    gap: 20px;
    margin: 20px 0;
    justify-content: center;
    flex-wrap: wrap;
}
.strategy-card {
    background: var(--jp-layout-color1, #f5f5f5);
    padding: 20px;
    border-radius: 8px;
    flex: 1 1 250px;
    min-width: 250px;
    max-width: 400px;
    box-sizing: border-box;
    border: 1px solid var(--jp-border-color2, rgba(0, 0, 0, 0.12));
}
@media (max-width: 600px) {
    .stat-cards-container {
        flex-direction: column;
        align-items: center;
    }
    .stat-card {
        width: 100%;
        max-width: 100%;
    }
    .strategy-cards-container {
        flex-direction: column;
        align-items: center;
    }
    .strategy-card {
        width: 100%;
        max-width: 100%;
    }
}
</style>
"""))


def stat_card_html(
    title: str, value: str, subtitle: str = "", color: str = "#73c0de"
) -> str:
    """Return HTML string for a stat card (used in container)."""
    return f"""
        <div class="stat-card">
            <div style="color: var(--jp-ui-font-color2, #666); font-size: 14px;">{title}</div>
            <div style="font-size: 32px; color: {color}; margin: 10px 0; font-weight: bold;">{value}</div>
            <div style="color: var(--jp-ui-font-color3, #888); font-size: 12px;">{subtitle}</div>
        </div>
    """


def stat_cards_row(cards_html: list[str]) -> HTML:
    """Create a responsive row of stat cards."""
    return HTML(f"""
        <div class="stat-cards-container">
            {"".join(cards_html)}
        </div>
    """)


def metric_row(label: str, value: str, color: str = None) -> str:
    """Generate HTML for a metric row in a table."""
    style = f"color: {color};" if color else ""
    return (
        f'<tr><td>{label}</td><td style="text-align: right; {style}">{value}</td></tr>'
    )


def strategy_card_html(name: str, result: InvestmentResult, color: str) -> str:
    """Return HTML string for a strategy result card."""
    ret_color = COLORS["success"] if result.total_return >= 0 else COLORS["danger"]
    return f"""
        <div class="strategy-card" style="border-left: 4px solid {color};">
            <h3 style="color: {color}; margin: 0 0 15px 0;">{name}</h3>
            <table style="width: 100%; color: var(--jp-ui-font-color1, #333);">
                {metric_row("Investi", f"{result.total_invested:,.0f} €")}
                {metric_row("Valeur Finale", f"{result.final_value:,.0f} €")}
                {metric_row("Rendement Total", f"{result.total_return_pct:+.2f}%", ret_color)}
                {metric_row("Annualisé", f"{result.annualized_return:.2f}%")}
                {metric_row("Drawdown Max", f"-{result.max_drawdown:.2f}%", COLORS["danger"])}
                {metric_row("Volatilité", f"{result.volatility:.2f}%")}
                {metric_row("Ratio de Sharpe", f"{result.sharpe_ratio:.2f}")}
            </table>
        </div>
    """


def strategy_cards_row(cards_html: list[str]) -> HTML:
    """Create a responsive row of strategy cards."""
    return HTML(f"""
        <div class="strategy-cards-container">
            {"".join(cards_html)}
        </div>
    """)

## 1. Les Données : Indice MSCI World (1972–2025)

Nous utilisons l'**indice MSCI World**, qui suit les actions de grandes et moyennes capitalisations dans 23 pays développés. C'est l'un des indices de référence les plus utilisés pour mesurer la performance des actions mondiales.

Cela nous donne **53 ans** d'historique de marché — incluant la stagflation des années 1970, le Lundi Noir de 1987, la bulle internet, la crise financière de 2008, le COVID-19, et tout le reste.

In [ ]:
def create_price_chart(prices, title):
    """Create a price chart using ipecharts."""
    if len(prices) > 2000:
        step = len(prices) // 2000
        prices_sampled = prices.iloc[::step]
    else:
        prices_sampled = prices

    dates = [d.strftime("%Y-%m-%d") for d in prices_sampled.index]
    values = [float(v) for v in prices_sampled.values]

    option = {
        "backgroundColor": "transparent",
        "title": {"text": title, "left": "center", "textStyle": {"width": "90%", "overflow": "truncate"}},
        "toolbox": {"feature": {"restore": {"title": "Réinitialiser"}}},
        "tooltip": {"trigger": "axis", "axisPointer": {"type": "cross"}},
        "xAxis": {"type": "category", "data": dates},
        "yAxis": {"type": "value", "name": "Prix"},
        "series": [
            {
                "name": title,
                "type": "line",
                "data": values,
                "smooth": True,
                "symbol": "none",
                "areaStyle": {"opacity": 0.15},
            }
        ],
        "grid": {"left": "15%", "right": "5%", "bottom": "20%", "top": "15%"},
        "dataZoom": [
            {"type": "slider", "xAxisIndex": 0, "start": 0, "end": 100, "bottom": "5%"}
        ],
    }

    chart = EChartsRawWidget(option=option, style={"height": "400px", "width": "100%"})
    display(chart)


PRICES = load_msci_world()
create_price_chart(PRICES, "Indice MSCI World (1972-2025)")

## 2. Comparaison sur une Période

Choisissez une date de début et un montant d'investissement pour voir comment chaque stratégie aurait performé. Cela simule ce qui se serait passé si vous aviez investi à ce moment précis.

**Comment lire les résultats :**
- **Lump Sum** investit la totalité du montant dès le premier jour
- **DCA** répartit les achats sur la durée spécifiée (par ex. 12 versements mensuels)
- Les deux stratégies conservent ensuite l'investissement jusqu'à la fin de l'horizon

In [ ]:
style = {"description_width": "180px"}
layout = Layout(width="100%")

investment_amount = FloatText(
    value=10000, description="Montant Total (€) :", step=1000, style=style, layout=layout
)
start_date_input = DatePicker(
    value=datetime.date(1990, 1, 1),
    description="Date de Début :",
    style=style,
    layout=layout,
)
investment_horizon = IntText(
    value=20, description="Horizon (années) :", style=style, layout=layout
)
dca_frequency_selector = Dropdown(
    options=[("Hebdomadaire", "weekly"), ("Mensuel", "monthly"), ("Trimestriel", "quarterly")],
    value="monthly",
    description="Fréquence DCA :",
    style=style,
    layout=layout,
)
dca_duration_input = IntText(
    value=12, description="Durée DCA (mois) :", style=style, layout=layout
)

run_button = Button(
    description="Comparer les Stratégies",
    button_style="primary",
    icon="chart-line",
    layout=Layout(width="100%", margin="10px 0"),
)
output_area = Output()

FREQ_LABELS = {"weekly": "Hebdo", "monthly": "Mensuel", "quarterly": "Trimestriel"}

def create_portfolio_chart(ls_result, dca_result, freq):
    """Create portfolio value comparison chart."""
    freq_label = FREQ_LABELS.get(freq, freq.capitalize())
    dates = [d.strftime("%Y-%m-%d") for d in ls_result.timeline.index]
    option = {
        "backgroundColor": "transparent",
        "title": {"text": "Comparaison de la Valeur du Portefeuille", "left": "center", "textStyle": {"width": "90%", "overflow": "truncate"}},
        "toolbox": {"feature": {"restore": {"title": "Réinitialiser"}}},
        "tooltip": {"trigger": "axis", "axisPointer": {"type": "cross"}},
        "legend": {
            "data": ["Lump Sum", f"DCA ({freq_label})", "DCA Montant Investi"],
            "bottom": 0,
        },
        "xAxis": {"type": "category", "data": dates, "boundaryGap": False},
        "yAxis": {"type": "value", "name": "Valeur du Portefeuille (€)"},
        "series": [
            {
                "name": "Lump Sum",
                "type": "line",
                "data": [float(v) for v in ls_result.timeline["value"]],
                "smooth": True,
                "symbol": "none",
            },
            {
                "name": f"DCA ({freq_label})",
                "type": "line",
                "data": [float(v) for v in dca_result.timeline["value"]],
                "smooth": True,
                "symbol": "none",
            },
            {
                "name": "DCA Montant Investi",
                "type": "line",
                "data": [float(v) for v in dca_result.timeline["invested"]],
                "smooth": True,
                "symbol": "none",
                "lineStyle": {"type": "dotted", "opacity": 0.6},
            },
        ],
        "grid": {"left": "15%", "right": "5%", "bottom": "20%", "top": "15%"},
        "dataZoom": [
            {"type": "slider", "xAxisIndex": 0, "start": 0, "end": 100, "bottom": "10%"}
        ],
    }
    return EChartsRawWidget(option=option, style={"height": "550px", "width": "100%"})


def run_single_comparison(btn):
    with output_area:
        clear_output(wait=True)

        try:
            total = investment_amount.value
            start = start_date_input.value.strftime("%Y-%m-%d")
            horizon = investment_horizon.value
            freq = dca_frequency_selector.value
            dca_months = dca_duration_input.value

            start_dt = pd.to_datetime(start)
            end_dt = start_dt + pd.DateOffset(years=horizon)
            end = end_dt.strftime("%Y-%m-%d")

            params_ls = BacktestParams(
                total_amount=total, start_date=start, end_date=end
            )
            params_dca = BacktestParams(
                total_amount=total,
                start_date=start,
                end_date=end,
                dca_frequency=freq,
                dca_periods=dca_months if freq == "monthly" else None,
            )

            ls_result = calculate_lump_sum(PRICES, params_ls)
            dca_result = calculate_dca(PRICES, params_dca)

            winner = (
                "Lump Sum" if ls_result.final_value > dca_result.final_value else "DCA"
            )
            diff = abs(ls_result.final_value - dca_result.final_value)
            diff_pct = abs(ls_result.total_return_pct - dca_result.total_return_pct)
            winner_color = COLORS["lump_sum"] if winner == "Lump Sum" else COLORS["dca"]

            winner_banner = HTML(f"""
                <div style="text-align: center; padding: 20px; background: var(--jp-layout-color1, #f5f5f5);
                            border-radius: 12px; border: 1px solid var(--jp-border-color2, rgba(0, 0, 0, 0.12)); margin-bottom: 20px;">
                    <span style="font-size: 18px; color: var(--jp-ui-font-color2, #666);">Gagnant :</span>
                    <h1 style="color: {winner_color}; margin: 5px 0;">{winner}</h1>
                    <span style="font-size: 14px; color: var(--jp-ui-font-color2, #666);">
                        de {diff:,.0f} € ({diff_pct:.2f}% de rendement en plus)
                    </span>
                </div>
            """)

            freq_label = FREQ_LABELS.get(freq, freq.capitalize())
            cards_row = strategy_cards_row([
                strategy_card_html("Lump Sum", ls_result, COLORS["lump_sum"]),
                strategy_card_html(f"DCA ({freq_label})", dca_result, COLORS["dca"]),
            ])

            display(
                HTML(
                    f'<h2 style="color: var(--jp-ui-font-color1, #333);">Résultats : MSCI World ({start} à {end})</h2>'
                )
            )
            display(winner_banner)
            display(cards_row)
            display(
                HTML(
                    '<h3 style="color: var(--jp-ui-font-color1, #333); margin-top: 30px;">Valeur du Portefeuille dans le Temps</h3>'
                )
            )
            display(create_portfolio_chart(ls_result, dca_result, freq))

        except Exception as e:
            display(
                HTML(f"""
                <div style="color: {COLORS["danger"]}; padding: 20px; background: rgba(227, 70, 113, 0.1); border-radius: 8px;">
                    <strong>Erreur :</strong> {str(e)}
                </div>
            """)
            )
            import traceback

            traceback.print_exc()


run_button.on_click(run_single_comparison)

inputs_panel = VBox(
    [
        HTML('<h3 style="color: #2E86AB; margin-top: 0;">Paramètres d\'Investissement</h3>'),
        investment_amount,
        start_date_input,
        investment_horizon,
        HTML('<h4 style="color: #A23B72; margin-top: 15px;">Paramètres DCA</h4>'),
        dca_frequency_selector,
        dca_duration_input,
        run_button,
    ],
    layout=Layout(
        padding="15px",
        max_width="450px",
        width="100%",
        border="1px solid var(--jp-border-color2, #ddd)",
        border_radius="8px",
    ),
)

placeholder_html = HTML("""
    <div style="min-height: 400px; display: flex; align-items: center; justify-content: center;
                color: var(--jp-ui-font-color3, #888); font-style: italic;">
        Cliquez sur "Comparer les Stratégies" pour voir les résultats
    </div>
""")
with output_area:
    display(placeholder_html)

results_panel = VBox(
    [output_area],
    layout=Layout(
        width="100%",
        min_height="450px",
        border="1px solid var(--jp-border-color2, #ddd)",
        border_radius="8px",
        padding="15px",
    ),
)

VBox(
    [
        HBox([inputs_panel], layout=Layout(justify_content="center", width="100%")),
        results_panel,
    ],
    layout=Layout(gap="20px"),
)

## 3. Backtest Historique

Une seule période ne raconte pas toute l'histoire. Et si vous aviez choisi le mauvais moment pour commencer ?

Cette section répond à la question : **« Sur toutes les dates de début possibles dans l'histoire, quelle stratégie gagne le plus souvent ? »**

Nous exécutons des centaines de backtests — un pour chaque date de début possible — pour voir à quel point chaque stratégie performe de manière constante. C'est ce qu'on appelle une **analyse par fenêtre glissante**, la même méthodologie utilisée dans la recherche académique.

In [ ]:
bt_style = {"description_width": "180px"}
bt_layout = Layout(width="100%")

bt_amount = FloatText(
    value=10000,
    description="Montant (€) :",
    step=1000,
    style=bt_style,
    layout=bt_layout,
)
bt_horizon = IntText(
    value=20, description="Horizon (années) :", style=bt_style, layout=bt_layout
)
bt_dca_months = IntText(
    value=12, description="Période DCA (mois) :", style=bt_style, layout=bt_layout
)

bt_button = Button(
    description="Lancer les Backtests",
    button_style="success",
    icon="play",
    layout=Layout(width="100%", margin="10px 0"),
)
bt_output = Output()


def create_backtest_bar_chart(results, horizon):
    """Create backtest bar chart."""
    dates = [d.strftime("%Y-%m") for d in results["start_date"]]
    values = [float(v) for v in results["difference_pct"]]
    bar_data = [
        {"value": v, "itemStyle": {"color": COLORS["lump_sum"] if w else COLORS["dca"]}}
        for v, w in zip(values, results["ls_wins"])
    ]
    option = {
        "backgroundColor": "transparent",
        "title": {
            "text": "LS vs DCA : Différence de Rendement",
            "subtext": f"Chaque barre = une période de {horizon} ans",
            "left": "center",
            "textStyle": {"width": "90%", "overflow": "truncate"},
        },
        "toolbox": {"feature": {"restore": {"title": "Réinitialiser"}}},
        "tooltip": {"trigger": "axis", "axisPointer": {"type": "shadow"}},
        "xAxis": {"type": "category", "data": dates, "axisLabel": {"rotate": 45}},
        "yAxis": {"type": "value", "name": "LS - DCA (%)"},
        "series": [{"name": "Avantage LS", "type": "bar", "data": bar_data}],
        "grid": {"left": "15%", "right": "5%", "bottom": "25%", "top": "20%"},
        "dataZoom": [
            {"type": "slider", "xAxisIndex": 0, "start": 0, "end": 100, "bottom": "5%"}
        ],
    }
    return EChartsRawWidget(option=option, style={"height": "450px", "width": "100%"})


def create_histogram_chart(results, horizon):
    """Create histogram chart."""
    ls_returns = results["ls_return_pct"].values
    dca_returns = results["dca_return_pct"].values
    all_returns = np.concatenate([ls_returns, dca_returns])
    min_val = np.floor(all_returns.min() / 10) * 10
    max_val = np.ceil(all_returns.max() / 10) * 10
    bins = np.linspace(min_val, max_val, 21)
    ls_counts, _ = np.histogram(ls_returns, bins=bins)
    dca_counts, _ = np.histogram(dca_returns, bins=bins)
    bin_labels = [f"{int(bins[i])}-{int(bins[i + 1])}%" for i in range(len(bins) - 1)]

    option = {
        "backgroundColor": "transparent",
        "title": {
            "text": f"Distribution des Rendements sur {horizon} Ans",
            "left": "center",
            "textStyle": {"width": "90%", "overflow": "truncate"},
        },
        "toolbox": {"feature": {"restore": {"title": "Réinitialiser"}}},
        "tooltip": {"trigger": "axis", "axisPointer": {"type": "shadow"}},
        "legend": {"data": ["Lump Sum", "DCA"], "bottom": 0},
        "xAxis": {
            "type": "category",
            "data": bin_labels,
            "name": "Rendement (%)",
            "axisLabel": {"rotate": 45},
        },
        "yAxis": {"type": "value", "name": "Périodes"},
        "series": [
            {"name": "Lump Sum", "type": "bar", "data": [int(c) for c in ls_counts]},
            {"name": "DCA", "type": "bar", "data": [int(c) for c in dca_counts]},
        ],
        "grid": {"left": "15%", "right": "5%", "bottom": "25%", "top": "15%"},
    }
    return EChartsRawWidget(option=option, style={"height": "400px", "width": "100%"})


def run_backtests(btn):
    bt_button.disabled = True

    with bt_output:
        clear_output(wait=True)

        try:
            total = bt_amount.value
            horizon = bt_horizon.value
            dca_months = bt_dca_months.value

            min_date = PRICES.index.min()
            max_date = PRICES.index.max()
            max_start = max_date - pd.DateOffset(years=horizon)

            if max_start <= min_date:
                print(f"Pas assez de données pour des backtests sur {horizon} ans.")
                bt_button.disabled = False
                return

            start_dates = pd.date_range(start=min_date, end=max_start, freq="MS")
            start_dates_str = [d.strftime("%Y-%m-%d") for d in start_dates]

            progress = IntProgress(
                value=0,
                min=0,
                max=len(start_dates_str),
                description="En cours :",
                bar_style="info",
                style={"bar_color": "#3fa266", "description_width": "80px"},
                layout=Layout(width="100%"),
            )
            progress_label = HTML(
                f'<span style="color: var(--jp-ui-font-color2, #666);">0 / {len(start_dates_str)} backtests</span>'
            )
            progress_container = VBox(
                [progress, progress_label],
                layout=Layout(margin="10px 0"),
            )
            display(progress_container)

            def update_progress(current, total_count):
                progress.value = current
                progress_label.value = f'<span style="color: var(--jp-ui-font-color2, #666);">{current} / {total_count} backtests</span>'

            results = run_multiple_backtests(
                prices=PRICES,
                total_amount=total,
                start_dates=start_dates_str,
                investment_horizon_years=horizon,
                dca_frequency="monthly",
                dca_duration_months=dca_months,
                progress_callback=update_progress,
            )

            clear_output(wait=True)

            if len(results) == 0:
                print("Aucun backtest valide n'a pu être exécuté.")
                return

            results["start_date"] = pd.to_datetime(results["start_date"])
            results["end_date"] = results["start_date"] + pd.DateOffset(years=horizon)

            ls_wins = results["ls_wins"].sum()
            dca_wins = len(results) - ls_wins
            ls_win_pct = (ls_wins / len(results)) * 100
            avg_diff = results["difference_pct"].mean()

            first_start = results["start_date"].min().strftime("%Y")
            last_start = results["start_date"].max().strftime("%Y")
            last_end = results["end_date"].max().strftime("%Y")

            display(
                HTML(
                    '<h2 style="color: var(--jp-ui-font-color1, #333);">Résultats des Backtests : MSCI World</h2>'
                )
            )
            display(
                HTML(f"""
                <p style="color: var(--jp-ui-font-color1, #333);"><b>{len(results)} périodes glissantes de {horizon} ans testées</b></p>
                <p style="color: var(--jp-ui-font-color2, #666); font-size: 13px;">
                    Dates de début : {first_start} à {last_start} | Dates de fin : {int(first_start) + horizon} à {last_end}
                </p>
            """)
            )

            cards = stat_cards_row([
                stat_card_html(
                    "Victoires Lump Sum",
                    f"{ls_win_pct:.1f}%",
                    f"{ls_wins} sur {len(results)}",
                    COLORS["lump_sum"],
                ),
                stat_card_html(
                    "Victoires DCA",
                    f"{100 - ls_win_pct:.1f}%",
                    f"{dca_wins} sur {len(results)}",
                    COLORS["dca"],
                ),
                stat_card_html(
                    "Avantage Moyen LS",
                    f"{avg_diff:+.1f}%",
                    "",
                    COLORS["success"] if avg_diff > 0 else COLORS["danger"],
                ),
            ])
            display(cards)

            display(create_backtest_bar_chart(results, horizon))
            display(create_histogram_chart(results, horizon))

        except Exception as e:
            print(f"Erreur : {e}")
            import traceback

            traceback.print_exc()
        finally:
            bt_button.disabled = False


bt_button.on_click(run_backtests)

bt_inputs_panel = VBox(
    [
        HTML('<h3 style="color: #2E86AB; margin-top: 0;">Paramètres du Backtest</h3>'),
        bt_amount,
        bt_horizon,
        bt_dca_months,
        HTML(
            '<p style="color: var(--jp-ui-font-color3, #888); font-size: 12px; margin: 10px 0;">Exécute des backtests à partir de chaque mois des données disponibles.</p>'
        ),
        bt_button,
    ],
    layout=Layout(
        padding="15px",
        max_width="450px",
        width="100%",
        border="1px solid var(--jp-border-color2, #ddd)",
        border_radius="8px",
    ),
)

bt_placeholder_html = HTML("""
    <div style="min-height: 400px; display: flex; align-items: center; justify-content: center;
                color: var(--jp-ui-font-color3, #888); font-style: italic;">
        Cliquez sur "Lancer les Backtests" pour voir les résultats
    </div>
""")
with bt_output:
    display(bt_placeholder_html)

bt_results_panel = VBox(
    [bt_output],
    layout=Layout(
        width="100%",
        min_height="450px",
        border="1px solid var(--jp-border-color2, #ddd)",
        border_radius="8px",
        padding="15px",
    ),
)

VBox(
    [
        HBox([bt_inputs_panel], layout=Layout(justify_content="center", width="100%")),
        bt_results_panel,
    ],
    layout=Layout(gap="20px"),
)

## Conclusion

**Le Lump Sum gagne environ deux tiers du temps.** Investir immédiatement bat l'investissement progressif dans environ 66% des périodes historiques. Cela correspond aux conclusions de [l'étude Vanguard de 2012](https://static.twentyoverten.com/5980d16bbfb1c93238ad9c24/rJpQmY8o7/Dollar-Cost-Averaging-Just-Means-Taking-Risk-Later-Vanguard.pdf), qui a analysé 85 ans de données américaines et plusieurs marchés internationaux.

Pourquoi ? Les marchés ont tendance à monter avec le temps. Le DCA laisse du cash non investi, manquant des gains potentiels. Plus votre période de DCA est longue, plus vous sacrifiez de rendements.

Mais le DCA n'est pas une erreur. Bien que le Lump Sum offre des rendements espérés plus élevés, le DCA offre un parcours plus régulier. Dans l'étude Vanguard, les portefeuilles Lump Sum ont connu des rendements négatifs sur 10 ans dans 22% des cas, contre seulement 18% pour le DCA. Si bien dormir compte plus que maximiser chaque euro, ce compromis peut en valoir la peine.

Le DCA a du sens si vous ne supportez pas d'investir tout d'un coup, si vous investissez de toute façon à partir de chaque salaire, ou si vous voulez vous protéger contre le regret d'un mauvais timing.

Si vous avez l'argent maintenant et un horizon long, les mathématiques favorisent l'investissement immédiat. Mais si le DCA est ce qui vous pousse à investir au lieu de rester en cash, c'est le meilleur choix *pour vous*. Les deux stratégies battent le fait de ne pas investir du tout.

*Les performances passées ne garantissent pas les résultats futurs. Ceci est à des fins éducatives uniquement, pas un conseil financier.*

---
*Données : Indice MSCI World (1972–2025) | Recherche : [Vanguard (2012)](https://static.twentyoverten.com/5980d16bbfb1c93238ad9c24/rJpQmY8o7/Dollar-Cost-Averaging-Just-Means-Taking-Risk-Later-Vanguard.pdf)*